In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import math
from scipy.spatial import distance
from timeit import default_timer as timer

#Parallelization Libaries
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()

In [2]:
#Load Data
#tilt = pd.read_csv('./tilt_frame=23.csv', header = None)
tilt = pd.read_csv('tilt_frame=23.csv', header = None)

In [3]:
#indexing of tilt
t_full = tilt.reset_index().values
t_index = t_full[:,0:3] #numpy.ndarray
print(t_index.shape)
#Parameters
t_row = len(tilt.index) #int
print(t_row)
b_tol = 2.5
t_tol = 22.5
t_bot = t_index[:,1:3] - b_tol
print(t_bot.shape)
t_top = t_index[:,1:3] + t_tol
print(t_index.shape)

(186067, 3)
186067
(186067, 2)
(186067, 3)


In [4]:
#Bott and Top tol
t_index = np.append(t_index,t_bot, axis = 1)
t_index = np.append(t_index,t_top, axis = 1)


In [5]:
#Index Sorting Function
## returns sorted row 
def sort(x, t_index):
    ss_index = np.full(shape = 100, fill_value = -1, dtype = float)
    x_bott = t_index[x,3]
    x_top  = t_index[x,5]
    y_bott = t_index[x,4]
    y_top  = t_index[x,6]
    ss_index[0] = t_index[x,1]
    ss_index[1] = t_index[x,2]
    t = 2
    for i in range(0,t_row):
        if (x_bott <= t_index[i,1] <= x_top) and (y_bott <= t_index[i,2] <= y_top):
            ss_index[t] = t_index[i,0] 
            t += 1


    # return np.transpose(s_index)
    

    new=np.delete(ss_index,np.where(ss_index==-1))
    x_b=t_index[x,1]-2.5
    x_t=t_index[x,1]+2.5
    y_b=t_index[x,2]-2.5
    y_t=t_index[x,2]+2.5
    s_index = np.full(shape = 27, fill_value = -1, dtype = float)
    s_index[0]=new[0]
    s_index[1]=new[1]

    t = 2
    for i in range (0,5):
        for k in range (0,5):
            for m in range (0,len(new)):
                if (x_b <= t_index[int(new[m]),1] <= x_t) and (y_b <= t_index[int(new[m]),2] <= y_t):
                    s_index[t]=new[m]
                    t+=1
                    break
    
            x_b+=5
            x_t+=5

        y_b+=5
        y_t+=5
        x_b=t_index[0,1]-2.5
        x_t=t_index[0,1]+2.5
    return s_index


In [6]:
sort(186067-1, t_index)

array([ 2.03900e+03,  1.53400e+03,  1.86062e+05, -1.00000e+00,
       -1.00000e+00, -1.00000e+00, -1.00000e+00, -1.00000e+00,
       -1.00000e+00, -1.00000e+00, -1.00000e+00, -1.00000e+00,
       -1.00000e+00, -1.00000e+00, -1.00000e+00, -1.00000e+00,
       -1.00000e+00, -1.00000e+00, -1.00000e+00, -1.00000e+00,
       -1.00000e+00, -1.00000e+00, -1.00000e+00, -1.00000e+00,
       -1.00000e+00, -1.00000e+00, -1.00000e+00])

In [ ]:
#Running
start = timer()
result = Parallel(n_jobs=num_cores)(delayed(sort)(x, t_index) for x in range(0, t_row) )
stop = timer()
print(stop-start)
#result =  Parallel(n_jobs=num_cores, backend="threading")(delayed(sort)(x, t_index) for x in range(0, t_row) );
#print(result)

In [ ]:
#Adds theta to end of index
tilt_t = t_full[:,3:28]
#Adds theta to end of index
s_index = np.array(result)
s_index = np.append(s_index, tilt_t, axis = 1)
s_index = s_index[(s_index != -1).all(axis=1)]
s_indexFull = s_index
tilt_t = s_index[:,27:52]
s_index = s_index[:,0:27]

In [ ]:
df = pd.DataFrame(s_index)
#df.to_csv('./index_full051619_frame=___.csv',header = None, index = None)
df.to_csv('index_fullParallelizationFull.csv',header = None, index = None)

In [ ]:
#load/set indexfull variable
indexFull = s_index

In [ ]:
#import Stress
#stress = pd.read_csv('stress_frame_x55_56_63_70_71_py=23.csv', header = None)
stress = pd.read_csv('stress_frame_x55_56_63_70_71_py=23.csv', header = None)
stress_5 = stress.reset_index().values #conver to numpy

In [ ]:
#Fill out full csv
start = timer()
m_row = len(indexFull)
s_full = indexFull[:,0:2]
temp = np.empty(shape = (m_row,125))
for y in range(0,m_row):
    r = 0
    for i in range(2,27):
        t_i = int(indexFull[y,i])
        temp[y,r:r+5] = stress_5[t_i,1:6]
        r += 5
s_full = np.append(s_full,temp, axis=1)
stop = timer()
print(stop-start)

In [ ]:
#Print out full csv
s_full = np.append(tilt_t,s_full, axis = 1)
df = pd.DataFrame(s_full)
df.to_csv("s_full_Parallelization.csv", header = None, index = None)